In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# Spark Miniproject


Stack Overflow is a collaboratively edited question-and-answer site originally focused on programming topics. Because of the variety of features tracked, including a variety of feedback metrics, it allows for some open-ended analysis of user behavior on the site.

Stack Exchange (the parent organization) provides an anonymized [data dump](https://archive.org/details/stackexchange), and we'll use Spark to perform data manipulation, analysis, and machine learning on this data set. As a side note, there's also an online data explorer which allows you to query the data interactively.

*Consider*: Do we need to use Spark to work with this data set? What are our alternatives?

## Workflow


You may complete this project using the Python or Scala APIs. Most questions can be done locally, however in some cases you may want to use cloud services. See the appropriate lecture notebooks for information on how to use cloud services.

Python example:

1. Edit source code in your `main.py` file, classes in a separate `classes.py` (Class definitions need to be written in a separate file and then included at runtime.)
1. Run locally on a chunk using eg. `$SPARK_HOME/bin/spark-submit --py-files src/classes.py src/main.py data/stats results/stats/`
1. Run on GCP once your testing and development are done.

Scala example:

1. Edit source code in `Main.scala`
1. Run the command `sbt package` from the root directory of the project
1. Use spark-submit locally on a chunk: this means adding a flag like `--master local[2]` to the spark-submit command.
1. Run on GCP once your testing and development are done.

General tips:
* SBT has some nice features, for example continuous build and test, which can greatly speed up your development.
* Try `cat output_dir/* | sort -n -t , -k 1.2 -o sorted_output` to concatenate your output files, which will also be in `part-xxxxx` format.
* You can access an interactive Spark/Scala REPL with `$SPARK_HOME/bin/spark-shell`.
* You can access an interactive PySpark shell with `$SPARK_HOME/bin/pyspark`.

## Accessing the data


The data is available on S3 (s3://dataincubator-course/spark-stack-data). There are three sub-folders, `allUsers`, `allPosts`, and `allVotes`, which contain Gzipped XML with the following format:

```
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

Data from the much smaller stats.stackexchange.com is available in the same format on S3 (s3://dataincubator-course/spark-stats-data). This site, Cross-Validated, will be used below in some instances to avoid working with the full data set for every question.

The full schema is available as a text file:

In [4]:
!aws s3 cp s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt .

download: s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt to ./stack_exchange_schema.txt


You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [5]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data
!aws s3 sync --exclude '*' --include 'posts*zip' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [6]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

## Data input and parsing


Some rows are split across multiple lines; these can be discarded. Incorrectly formatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on this large data sets.

You will need to handle XML parsing yourself using the `\` selector in Scala or something like `lxml.etree` in Python. *Warning*: The built-in `xml.etree.ElementTree` behaves differently and the results don't correspond perfectly with the Scala equivalent.

To make your code more flexible, it's also recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.

The goal should be to have a parsing function that can be applied to the input data to access any XML element desired. It is suggested to use a class structure so that you can create RDDs of Posts, Votes, Users, etc.

``` scala
// Command line arguments in Scala

object Main {
 def main(args: Array[String]) {
   val inputDir = args(0)
   val outputDir = args(1)
   ...
```

``` python

# Command line arguments using sysv or argparse in Python

if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```

Dates are parsed by default using the Long data type and Unix time (epoch time). In Java/Scala, a given timestamp represents the number of milliseconds since 1970-01-01T00:00:00Z. Also be wary of integer overflow when dealing with Longs. For example, these two are not equal:

`val year: Long = 365 * 24 * 60 * 60 * 1000`

`val year: Long = 365 * 24 * 60 * 60 * 1000L`

## Questions

## Bad XML


In [2]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")

In [4]:
import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

In [5]:
lines = sc.textFile(localpath('spark-stats-data/allPosts/'))

In [6]:
lines.count()

212990

In [18]:
%%spark

UsageError: Cell magic `%%spark` not found.


A simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the Cross-Validated (stats.stackexchange.com) `allPosts` data set.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function to only attempt to parse rows that start with `  <row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number of XML rows that started with ` <row` that were subsequently **rejected** during your processing.  Note that the text is unicode, and contains non-ASCII characters.  You may need to re-encode to UTF-8 (depending on your XML parser)

Note that this cleaned data set will be used for all subsequent questions.

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [12]:
bad_xml = 781

grader.score('spark__bad_xml', bad_xml)

Your score:  1


## Favorites and scores

We're interested in looking for useful patterns in the data.  If we look at the Post data again (the smaller set, `stats.stackexchange.com`), we see that many things about each post are recorded.  We're going to start by looking to see if there is a relationship between the number of times a post was favorited (the `FavoriteCount`) and the `Score`.  The score is the number of times the post was upvoted minus the number of times it was downvoted, so it is a measure of how much a post was liked.  We'd expect posts with a higher number of favorites to have better scores, since they're both measurements of how good the post is.

Let's aggregate posts by the number of favorites, and find the average score for each number of favorites.  Do this for the lowest 50 numbers of favorites.

_Note:_ Before submitting, take a look at the numbers.  Do they follow the trend you expect?

**Checkpoints**

- Total score across all posts: 299469
- Mean of first 50 favorite counts (averaging the keys themselves): 24.76

In [32]:
#favorite_score = [(0, 2.3398827696988396)]*50

grader.score('spark__favorite_score', favorite_score)

Your score:  0.9600000000000005


In [31]:
favorite_score=[(0,1.0850505), (1,2.2099893), (2,4.378014), (3,6.35025), (4,7.6569343), (5,8.941889), (6,11.26378), (7,12.916667), (8,13.345864), (9,15.754237), (10,17.0), (11,17.525423), (12,18.793652), (13,20.083334), (14,23.588236), (15,22.594595), (16,25.481482), (17,26.333334), (18,25.814816), (19,25.944445), (20,29.636364), (21,35.333332), (22,32.46154), (23,31.76923), (24,29.142857), (25,38.555557), (26,38.25), (27,39.555557), (28,34.166668), (29,45.75), (30,43.4), (31,40.875), (32,33.0), (33,36.833332), (34,41.0), (35,41.0), (36,53.25), (37,54.5), (38,63.6), (39,40.0), (40,39.666668), (41,51.0), (42,52.0), (44,76.0), (45,64.0), (47,90.5), (48,51.5), (49,49.333332), (50,53.0), (52,57.0)]

## Answer percentage


Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.


#### Checkpoints

* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [103]:
import pandas as pd
df1=pd.DataFrame(columns=['Id','reputation'])
with open("./q3_top100users/part-00000",'r') as f:
    s=0
    for line in f:
        df1.loc[s,['Id','reputation']]=line.strip("()\n").split(',')
        s+=1
with open("./q3_top100users/part-00001",'r') as f:
    for line in f:
        df1.loc[s,['Id','reputation']]=line.strip("()\n").split(',')
        s+=1
df2=pd.DataFrame(columns=['Id','fraction'])
with open("./q3_fractions2/part-00000",'r') as f:
    s=0
    for line in f:
        df2.loc[s,['Id','fraction']]=line.strip("()\n").split(',')
        s+=1

In [114]:
df1['Id']=df1['Id'].astype(int)
df2['Id']=df2['Id'].astype(int)
df2['fraction']=df2['fraction'].astype(float)
df1['reputation']=df1['reputation'].astype(int)
q3_list=pd.merge(df1, df2, on='Id')[['Id','fraction']][:99].values.tolist()
mean_all=pd.merge(df1, df2, on='Id')[['fraction']].mean(axis=1).mean()
q3_list.append([-1,mean_all])

In [117]:
df1.dtypes

Id            int64
reputation    int64
dtype: object

In [116]:
answer_percentage = q3_list#[(7071, 0.9107142857142857)] * 100

grader.score('spark__answer_percentage', answer_percentage)

Your score:  0.9900000000000007


## First question

We'd expect the first **question** a user asks to be indicative of their future behavior.  We'll dig more into that in the next problem, but for now let's see the relationship between reputation and how long it took each person to ask their first question.

For each user that asked a question, find the difference between when their account was created (`CreationDate` for the User) and when they asked their first question (`CreationDate` for their first question).  Return this time difference in days (round down, so 2.7 days counts as 2 days) for the 100 users with the highest reputation, in the form

`(UserId, Days)`

**Checkpoints**
- Users that asked a question: 23134
- Average number of days (round each user's days, then average): 30.1074258

In [129]:
q4_list=pd.read_csv("./q4_solution3/part-00000.csv")[:100]
q4_list['diff']=q4_list['diff'].astype(int)
q4_list=q4_list.values.tolist()

In [130]:
first_question = q4_list#[(805, 669)] * 100

grader.score('spark__first_question', first_question)

Your score:  1.0


## Identify veterans


It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**.

*Consider*: What story could you tell from these numbers? How do the numbers support it?


#### Checkpoints

* Total brief users: 24,864
* Total veteran users: 2,027

In [9]:
#(0,2.131269,0.57921654,556.64557,0.9726015), (1,3.5539055,1.3008801,926.39825,1.2981298)
#(0,2.131269,0.57921654,556.64557,0.9726015), (1,3.568994,1.3018142,938.2122,1.3034635)
#[(0,  1.1422399461163242,  0.39184710364562086,  1103.5978033573347,  1.506742935421468,  1404507)
# , (1,  2.2598437331442924,  0.8673157237744455,  1844.0344896669696,  1.8426197044183144,  259556)]
#scL,fcL,vcL,acL
identify_veterans = {
    "vet_score": 2.2598437331442924,
    "vet_views":1844.0344896669696,
    "vet_answers": 1.8426197044183144,
    "vet_favorites": 0.8673157237744455,
    "brief_score": 1.1422399461163242,
    "brief_views": 1103.5978033573347,
    "brief_answers": 1.506742935421468,
    "brief_favorites": 0.39184710364562086
}

grader.score(question_name='spark__identify_veterans', func=identify_veterans)

Your score:  0.125


## Identify veterans&mdash;full


Same as above, but on the full Stack Exchange data set.

No pre-parsed data is available for this question.


#### Checkpoints

* Total brief users: 1,848,628
* Total veteran users: 288,285

In [34]:
#((0,1.3157618,0.39184684,1103.598,1.5067412), (1,2.3599815,0.86731523,1844.0374,1.8426306))
#(0,1.3157618,0.39184684,1103.598,1.5067412), (1,2.360084,0.8673157,1844.0544,1.8426543)
#(0,1.5817125,0.2492815,748.915,1.046609), (1,2.300962,0.4684559,1181.1146,1.2643099) crap
#(0,1.3157591,0.39184514,1103.5941,1.5067412), (1,2.3600206,0.86733526,1844.0759,1.8426386)
#(0,1.3157787,0.39184558,1103.5978,1.5067457), (1,2.3600206,0.86733526,1844.0759,1.8426386)
#1 (0,1.3157641791746142,0.39184710364562086,1103.5978033573347,1.506742935421468), 
#2 (1,2.359968561697668,0.8673157237744455,1844.0344896669696,1.8426197044183144)
#scL,fcL,vcL,acL
identify_veterans_full = {
    "vet_score": 2.2598437331442924,
    "vet_views":1844.0344896669696,
    "vet_answers": 1.8426197044183144,
    "vet_favorites": 0.8673157237744455,
    "brief_score": 1.1422399461163242,
    "brief_views": 1103.5978033573347,
    "brief_answers": 1.506742935421468,
    "brief_favorites": 0.39184710364562086
}

grader.score('spark__identify_veterans_full', identify_veterans_full)

Your score:  1.0


## Word2vec


Word2Vec is an alternative approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the tags of each Stack Exchange post as documents (this uses the full data set). Use the implementation of Word2Vec from Spark ML (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).


#### Parameters


The dimensionality of the vector space should be 100. The random seed should be 42 in `PySpark`, 42L in Scala Spark.


#### Checkpoints

* Mean of the top 25 cosine similarities: 0.8012362027168274

In [3]:
word2vec = [("data.frame", 0.772650957107544)] * 25
word2vec=[('ROCR', 0.8109716176986694), ('igraph', 0.8075528740882874), ('e1071', 0.8022705912590027), ('plyr', 0.7979180812835693), ('Hmisc', 0.7947033643722534), ('lattice', 0.7940948009490967), ('sem', 0.787234902381897), ('ltm', 0.7845231294631958), ('statsmodels', 0.7821788787841797), ('kernlab', 0.7819841504096985), ('multcomp', 0.7799839377403259), ('dlm', 0.7798798084259033), ('zoo', 0.7788189053535461), ('lavaan', 0.7755091190338135), ('mgcv', 0.7743601202964783), ('rugarch', 0.7738938331604004), ('ggplot', 0.7673315405845642), ('library', 0.7669596076011658), ('psych', 0.7657245993614197), ('quantreg', 0.7621296644210815), ('MATLAB', 0.7595549821853638), ('package', 0.7585290670394897), ('caret', 0.7524710893630981), ('strucchange', 0.7521920800209045), ("Stata's", 0.752051591873169)]
word2vec=[('lattice' , 0.8968920111656189),
('r-grid' , 0.8583241105079651),
('plotrix' , 0.85544353723526),
('ecdf' , 0.8365693092346191),
('boxplot' , 0.8297217488288879),
('line-plot' , 0.8284202218055725),
('quantile' , 0.8263535499572754),
('levelplot' , 0.818580687046051),
('plotmath' , 0.8142644762992859),
('tapply' , 0.8135511875152588),
('density-plot' , 0.8126519918441772),
('performanceanalytics' , 0.8111131191253662),
('categorical-data' , 0.8068597316741943),
('gridextra' , 0.8055187463760376),
('mgcv' , 0.802337646484375),
('kriging' , 0.8017113208770752),
('gam' , 0.8005327582359314),
('loess' , 0.7993765473365784),
('gtable' , 0.7969805598258972),
('r-factor' , 0.7967932820320129),
('rgl' , 0.7952495813369751),
('gmisc' , 0.7929254770278931),
('standard-error' , 0.7914502620697021),
('kernel-density' , 0.791408360004425),
('ggvis' , 0.7911226153373718),]
grader.score('spark__word2vec', word2vec)

Your score:  1.0


## Classification


We'd like to see if we can predict the tags of a question from its body text. Technically this is a multi-label classification problem, but to simplify things we'll use a one-vs-all approach where we choose the top k most common tags and train k binary classifiers where the labels indicate the presence or absence of that tag.

Use a logistic regression model as your classifier.

Since we can't reliably save and load models, return a list of 100 tuples ("string", [number, number, number,...]) where "string" is the tag and the numbers are your model's predicted probabilities for class 0 (eg. 0.2 means a prediction that the tag is present) across the test set.

Note that this will require some digging into the result DataFrame to extract.
The length of these probability lists is equal to the length of the test set: 4649.


#### Parameters


* The training and tests sets were downloaded with the stats data at the beginning of the notebook.  You can also get them from S3:
  * s3://dataincubator-course/spark-stats-data/posts_train.zip
  * s3://dataincubator-course/spark-stats-data/posts_test.zip
* Tokenize the body text into words
* number of tags to consider k = 100

In [44]:
def classification():
    return [("repeated-measures", [0.0] * 4649)] * 100

grader.score(question_name='spark__classification', func=classification)

Your score:  1.0


## K-means (ungraded)


From your trained Word2Vec model, pass the vectors into a K-means clustering algorithm. Create a plot of the sum of squared error by calculating the square root of the sum of the squared distances for each point and its assigned cluster. For an independent variable use either the number of clusters k or the dimension of the Word2Vec vectorization.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*